Part 1 - Implementation

In [1]:
import numpy as np

In [2]:
# Model Parameters
input_size = 2 # no. of features
layers = [4,3] # no. of neurons in 1st and 2nd layer
output_size = 2

In [3]:
class NeuralNetwork:
    def __init__(self,input_size,layers,output_size):
        np.random.seed(0)

        model = {}

        # First Layer
        model['W1'] = np.random.randn(input_size,layers[0])
        model['b1'] = np.zeros((1,layers[0]))

        # Second Layer
        model['W2'] = np.random.randn(layers[0],layers[1])
        model['b2'] = np.zeros((1,layers[1]))
        
        # Third Layer
        model['W3'] = np.random.randn(layers[2],output_size)
        model['b3'] = np.zeros((1,output_size))

        self.model = model
        